In [ ]:
!pip install pyrsgis
!pip install pygad

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 KB 6.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import math
from copy import deepcopy
from pyrsgis import raster
from skimage.metrics import structural_similarity as ssim
from pygad import GA
from osgeo import gdal
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
f2017 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/2017.tif"
f2020 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/2020.tif"
road_network_path = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/road.tif"
pop = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/pop.tif"
metro = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/metro.tif"

ds1, arr1 = raster.read(f2017,bands=1)
ds2,arr2 = raster.read(f2020,bands =1)
road_ds,road_arr = raster.read(road_network_path,bands=1)
pop_ds, pop_arr = raster.read(pop,bands=1)
metro_ds,metro_arr = raster.read(metro,bands=1)


if(ds1.RasterXSize == ds2.RasterXSize and ds1.RasterYSize == ds2.RasterYSize and ds1.RasterYSize == road_ds.RasterYSize and  ds1.RasterXSize == road_ds.RasterXSize):
  print("Input File size matched\n")
else:
  print("Input Files are not of same size\n")
  exit()


Input File size matched



In [ ]:
def fitness_function(solution,solution_idx):
    builtUpThreshold = solution[0]
    distFromRoad = solution[1]
    popThreshold = solution[2]
    metroThreshold = solution[3]


    kernelSize = 3
    predicted_arr = deepcopy(arr1)
    sideMargin = math.ceil(kernelSize/2)

    for y in range(sideMargin,no_rows-(sideMargin-1)):
        for x in range(sideMargin,no_cols-(sideMargin-1)):
            kernel = arr1[y-(sideMargin-1):y+(sideMargin), x-(sideMargin-1):x+(sideMargin)]
            builtupCount = sum(sum(kernel==1))
            if (builtupCount >= builtUpThreshold and road_arr[y,x]<=distFromRoad and arr1[y,x]!=3 and pop_arr[y,x]>=popThreshold and metro_arr[y,x]>=metroThreshold):
                predicted_arr[y,x]=1


    return -ssim(arr2,predicted_arr,data_range=4)


# Define the initial GA parameters
initial_algo_param = {'max_num_iteration': 10,
                      'population_size': 30,
                      'mutation_probability': 0.1,
                      'elit_ratio': 0.01,
                      'crossover_probability': 0.5,
                      'parents_portion': 0.3,
                      'crossover_type': 'uniform',
                      'max_iteration_without_improv': None}

In [ ]:
#Create an instance of GA-class
ga_instance = GA (num_generations= 10,
                  num_parents_mating= int(initial_algo_param['population_size']*initial_algo_param['parents_portion']),
                  fitness_func= fitness_function,

                  sol_per_pop= 30,
                  num_genes=4,

                  init_range_low=  -100,
                  init_range_high= 1000,
                  parent_selection_type='sss',

                  mutation_type = "adaptive",
                  mutation_probability=[0.6,0.2])

#Run the GA
ga_instance.run()

ga_instance.plot_result()

In [ ]:
best_solution, best_fitness = ga_instance.best_solution()

print("Best solution:", best_solution)
print("Best fitness value:", best_fitness)

In [ ]:
no_rows = ds1.RasterYSize
no_cols = ds1.RasterXSize
kernelSize = 3
predicted_arr = deepcopy(arr1)
sideMargin = math.ceil(kernelSize/2)

builtUpThreshold = best_solution[0]
distFromRoad = best_solution[1]
popThreshold = best_solution[2]
metroThreshold = best_solution[3]

for y in range(sideMargin,no_rows-(sideMargin-1)):
    for x in range(sideMargin,no_cols-(sideMargin-1)):
        kernel = arr1[y-(sideMargin-1):y+(sideMargin), x-(sideMargin-1):x+(sideMargin)]
        builtupCount = sum(sum(kernel==1))
        if (builtupCount >= builtUpThreshold and road_arr[y,x]<=distFromRoad and arr1[y,x]!=3):
            predicted_arr[y,x]=1

def exportPredicted(outFileName):
    driver = gdal.GetDriverByName("GTiff")
    outdata = driver.Create(outFileName, no_cols, no_rows, 1, gdal.GDT_UInt16) # option: GDT_UInt16, GDT_Float32
    outdata.SetGeoTransform(ds1.GetGeoTransform())
    outdata.SetProjection(ds1.GetProjection())
    outdata.GetRasterBand(1).WriteArray(predicted_arr)
    outdata.GetRasterBand(1).SetNoDataValue(0)
    outdata.FlushCache()
    outdata = None

print("SSIM:", ssim(arr2,predicted_arr,data_range=4))
exportPredicted("Bangalore_AMGA_2020.tif")